In [ ]:
#| default_exp vanity

In [ ]:
#| hide

from nostr_relay import web

In [ ]:
#| hide

web.run_with_uvicorn(conf_file='../nostr-relay/nostr-relay-config.yml', in_thread=True)

2023-01-11 15:45:24,733 - nostr_relay.web - INFO - Starting version 1.3.4


INFO:     Started server process [81343]
INFO:     Waiting for application startup.


# vanity

> Use the notifyr bot to generate vanity addresses and get notified by DM when they are done.

## Finding vanity addresses
Here is [an interesting article by Kris Constable about vanity addresses](https://krisconstable.com/generating-a-key-pair-with-nostr/). This is a great usecase for our notifyr decorator because calculating vanity addresses can be quite slow if we are looking for long words.

This module will let you look for an `npub` or a `hex` vanity address that starts with a particular pattern.

In [ ]:
#| export

import time
import secrets
import secp256k1
import functools
from typing import Union
from nostrfastr.nostr import PrivateKey
from nostrfastr.notifyr import notifyr
from nostr import bech32

In [ ]:
#| export

def _guess_bytes():
    privkey_bytes = secrets.token_bytes(32) 
    sk = secp256k1.PrivateKey(privkey_bytes)
    pubkey_bytes = sk.pubkey.serialize()[1:]
    return privkey_bytes, pubkey_bytes

def _make_bech32(pubkey_bytes):
    converted_bits = bech32.convertbits(pubkey_bytes, 8, 5)
    pubkey = bech32.bech32_encode('npub', converted_bits, bech32.Encoding.BECH32)
    return pubkey

def _make_hex(pubkey_bytes):
    return pubkey_bytes.hex()

## making guesses
In this module you will find a `guess_npub` and `guess_hex` methods that are created from `_guess_vanity`. We are going directly back to the bits to get the best perfomance possible - this code is very similar to the code used to generate private keys in the `nostr.PrivateKey` class from `python-nostr`

In [ ]:
#| export

def _guess_vanity(make_format, startswith=''):
    privkey_bytes, pubkey_bytes = _guess_bytes()
    pubkey_hex = make_format(pubkey_bytes)
    if pubkey_hex.startswith(startswith):
        return privkey_bytes.hex(), pubkey_hex
    else:
        return None, None


In [ ]:
#| export

guess_bech32 = functools.partial(_guess_vanity, make_format=_make_bech32)
guess_hex = functools.partial(_guess_vanity, make_format=_make_hex)


In [ ]:
privkey_hex, npub = guess_bech32()
assert PrivateKey.from_hex(privkey_hex).public_key.bech32() == npub

In [ ]:
privkey_hex, pubkey_hex = guess_hex()
assert PrivateKey.from_hex(privkey_hex).public_key.hex() == pubkey_hex

## checking computer hashrate
We can make some functions to check the guess rate (hashrate)

In [ ]:
#| export

def _time_guess(guesser):
    """get a timed assessment of a guess

    Parameters
    ----------
    guesser : function
        either `guess_npub` or `guess_hex`

    Returns
    -------
    float
        time in seconds
    """
    start = time.perf_counter()
    pub = guesser(startswith=' ')
    if pub is None:
        pass
    end = time.perf_counter()
    interval = end - start
    return interval

def _get_guess_time(guesser, n_guesses=1e4):
    """estimate a guess rate

    Parameters
    ----------
    guesser : function
        either `guess_npub` or `guess_hex`
    n_guesses : float, optional
        number of guesses to make for estimation, by default 1e4

    Returns
    -------
    float
        time in seconds
    """
    n_guesses = int(n_guesses)
    t = sum([_time_guess(guesser) for _ in range(n_guesses)]) / n_guesses
    return t

In [ ]:
f'We estimage a hash rate of {1/_get_guess_time(guess_hex)} guesses per second'

'We estimage a hash rate of 9750.069930489984 guesses per second'

In [ ]:
f'We estimage a hash rate of {1/_get_guess_time(guess_bech32)} guesses per second'

'We estimage a hash rate of 4548.230563135903 guesses per second'

And use our hashrate to make some estimates of how quickly we can find certain vanity addresses

In [ ]:
#| export

import math

In [ ]:
#| export

def _expected_guesses_by_char(options: Union[str,list], num_char: int) -> float:
    """return an average number of guesses it would take to guess
    a pattern based on the number of characters in the pattern and
    the number of character options in the random output

    Parameters
    ----------
    options : list or str
        a set of characters as a str or list that are options for
        guessing
    num_char : int
        the number of characters in the pattern

    Returns
    -------
    float
        the expected number of guesses required to match the pattern
    """
    p = 1 / len(options)
    return (p ** -num_char - 1)/ (1 - p)

def _expected_chars_by_time(options: Union[str,list], num_guesses: int) -> float:
    """the length of pattern you might expect to be able to guess given a
    certain number of guesses.

    Parameters
    ----------
    options : list or str
        a set of characters as a str or list that are options for
        guessing
    num_guesses : int
        the total number of guesses at a pattern

    Returns
    -------
    float
        th
    """
    p = 1 / len(options)
    n = - math.log(1 + (num_guesses * (1 - p))) / math.log(p)
    return n

def _expected_time(options: Union[str,list], num_char: int, time_per_guess: float) -> float:
    """the expected amount of time it would take to guess a pattern with a certain
    length based on the average time per guess and the character options

    Parameters
    ----------
    options : list or str
        a set of characters as a str or list that are options for
        guessing
    num_char : int
        the number of characters in the pattern
    time_per_guess : float
        averge time per guess in seconds

    Returns
    -------
    float
        the expected amount of time needed to guess the pattern
    """
    n_guess = _expected_guesses_by_char(options, num_char)
    time_seconds = n_guess * time_per_guess
    return time_seconds

hex_chars = 'abcdef0123456789'
npub_chars = '023456789acdefghjklmnpqrstuvwxyz'


In [ ]:
#| export

def _average_char_by_time(options: Union[str,list], time_per_guess: float) -> None:
    """print an average number of characters you would expect to be
    able to guess for certain time periods based on character options
    and the average time per guess

    Parameters
    ----------
    options : list or str
        a set of characters as a str or list that are options for
        guessing
    time_per_guess : float
        the average time elapsed per guess
    """
    seconds_in_month = 60 * 60 * 24 * 30.5
    seconds_in_day = 60 * 60 * 24
    seconds_in_hour = 60 * 60
    seconds_in_minute = 60

    guesses_per_month = seconds_in_month / time_per_guess
    guesses_per_day = seconds_in_day / time_per_guess
    guesses_per_hour = seconds_in_hour / time_per_guess
    guesses_per_minute = seconds_in_minute / time_per_guess
    guesses_per_second = 1 / time_per_guess

    guesses = [guesses_per_second, guesses_per_minute,\
               guesses_per_hour, guesses_per_day, guesses_per_month]

    expected_chars = [_expected_chars_by_time(options, g) for g in guesses]
    results = zip(['one second', 'one minute', 'one hour', 'one day', 'one month'],
                   expected_chars)
    for t, c in results:
        print(f'In {t} you can expect to get {c} characters on average')

def _average_time_by_char(options: Union[str,list], time_per_guess: float) -> None:
    """print an average elapsed time for a range of pattern lengths

    Parameters
    ----------
    options : Union[str,list]
        a set of characters as a str or list that are options for
        guessing
    time_per_guess : float
        the average time elapsed per guess
    """
    for n in range(20):
        n += 1
        t = _expected_time(options, n, time_per_guess)
        print(f'{n} characters: it might take {t} seconds')



In [ ]:
#| hide

time_per_guess = _get_guess_time(guess_bech32)
_average_char_by_time(npub_chars, time_per_guess)

In one second you can expect to get 2.3873073739084365 characters on average
In one minute you can expect to get 3.568613098769878 characters on average
In one hour you can expect to get 4.749990011166678 characters on average
In one day you can expect to get 5.66698249171011 characters on average
In one month you can expect to get 6.65312995839842 characters on average


In [ ]:
#| hide

time_per_guess = _get_guess_time(guess_hex)
_average_char_by_time(hex_chars, time_per_guess)

In one second you can expect to get 3.303549094982759 characters on average
In one minute you can expect to get 4.780234421593435 characters on average
In one hour you can expect to get 6.256956448424652 characters on average
In one day you can expect to get 7.403197063500674 characters on average
In one month you can expect to get 8.635881397466482 characters on average


In [ ]:
#| hide

_average_time_by_char(hex_chars, time_per_guess)

1 characters: it might take 0.0015785832211840899 seconds
2 characters: it might take 0.026835914760129528 seconds
3 characters: it might take 0.43095321938325654 seconds
4 characters: it might take 6.896830093353288 seconds
5 characters: it might take 110.3508600768738 seconds
6 characters: it might take 1765.6153398132021 seconds
7 characters: it might take 28249.847015594452 seconds
8 characters: it might take 451997.5538280945 seconds
9 characters: it might take 7231960.8628280945 seconds
10 characters: it might take 115711373.8068281 seconds
11 characters: it might take 1851381980.910828 seconds
12 characters: it might take 29622111694.57483 seconds
13 characters: it might take 473953787113.19885 seconds
14 characters: it might take 7583260593811.184 seconds
15 characters: it might take 121332169500978.94 seconds
16 characters: it might take 1941314712015663.0 seconds
17 characters: it might take 3.106103539225061e+16 seconds
18 characters: it might take 4.969765662760097e+17 seco

In [ ]:
#| export

def expected_performance():
    print(
        '''This is a random guessing process - estimations are an average, but the actual
        time it takes to find a key could be significantly more or less than the estimate!
        Please keep that in mind when choosing an option.
        ''')
    print('hex:')
    time_per_guess_hex = _get_guess_time(guesser=guess_hex)
    _average_char_by_time(hex_chars, time_per_guess_hex)
    print('\n')
    _average_time_by_char(hex_chars, time_per_guess_hex)
    print('\n')

    print('npub:')
    time_per_guess_bech32 = _get_guess_time(guesser=guess_bech32)
    _average_char_by_time(npub_chars, time_per_guess_bech32)
    print('\n')
    _average_time_by_char(npub_chars, time_per_guess_bech32)
    print('\n')

    

In [ ]:
expected_performance()

This is a random guessing process - estimations are an average, but the actual
        time it takes to find a key could be significantly more or less than the estimate!
        Please keep that in mind when choosing an option.
        
hex:
In one second you can expect to get 3.2801263443168422 characters on average
In one minute you can expect to get 4.756809166584342 characters on average
In one hour you can expect to get 6.233531151671968 characters on average
In one day you can expect to get 7.379771766069949 characters on average
In one month you can expect to get 8.612456100007245 characters on average


1 characters: it might take 0.0016845129386987537 seconds
2 characters: it might take 0.028636719957878812 seconds
3 characters: it might take 0.45987203226475976 seconds
4 characters: it might take 7.359637029174855 seconds
5 characters: it might take 117.75587697973637 seconds
6 characters: it might take 1884.0957161887206 seconds
7 characters: it might take 30145.53314353247 

Lets make the function...

In [ ]:
#| export

def gen_vanity_pubkey(startswith: str, style='hex') -> PrivateKey:
    """randomly generate private keys until one matches the desire
    startswith for an npub or hex

    Parameters
    ----------
    startswith : str
        characters that the public key should start with. More chars
        means longer run time
    style : str, optional
        'npub' or 'hex' - npub is more commonly displayed on apps
        while hex is the true base private key with no encoding,
        by default 'hex'

    Returns
    -------
    PrivateKey
        returns a private key object
    """
    pubkey = None
    if style == 'npub':
        if not all(c in npub_chars for c in startswith):
            raise ValueError(f'character of selection not '
                              'in npub pattern ({npub_chars})')
        time_per_guess = _get_guess_time(guess_bech32)
        t = _expected_time(npub_chars, len(startswith), time_per_guess)
        startswith = f'npub1{startswith}'
    else:
        if not all(c in hex_chars for c in startswith):
            raise ValueError(f'character of selection not in '
                              'hex pattern ({hex_chars})')
        time_per_guess = _get_guess_time(guess_hex)
        t = _expected_time(hex_chars, len(startswith), time_per_guess)
    print(f'It might take {int(t)} seconds to find a {style} pubkey that starts with '
          f'{startswith}. Note that this is a very rough estimate and due '
          'to the random nature of finding vanity keys it could take MUCH '
          'longer.')
    while pubkey is None:
        if style == 'npub':
            privkey_hex, pubkey = guess_bech32(startswith=startswith)
        else:
            privkey_hex, pubkey = guess_hex(startswith=startswith)
    return PrivateKey.from_hex(privkey_hex)

In [ ]:
from fastcore.test import test_fail

Make sure we don't allow characters that will never happen - test that these cases fail

In [ ]:
fail = lambda _: gen_vanity_pubkey(startswith='b', style='npub')
test_fail(fail)
fail = lambda _: gen_vanity_pubkey(startswith='g', style='hex')
test_fail(fail)


Generate a couple npubs!

In [ ]:
vanity_private_key_npub = gen_vanity_pubkey(startswith='23', style='npub')
vanity_private_key_hex = gen_vanity_pubkey(startswith='23', style='hex')

It might take 0 seconds to find a npub pubkey that starts with npub123. Note that this is a very rough estimate and due to the random nature of finding vanity keys it could take MUCH longer.
It might take 0 seconds to find a hex pubkey that starts with 23. Note that this is a very rough estimate and due to the random nature of finding vanity keys it could take MUCH longer.


And make sure it worked...

In [ ]:
assert vanity_private_key_npub.public_key.bech32().startswith('npub123')
assert vanity_private_key_hex.public_key.hex().startswith('23')

Now we can also make a version of this that notifies you.

In [ ]:
#| export

vanity_notifyr = notifyr(gen_vanity_pubkey)

Remember that if you want a vanity notifyr that will go to a different address than the one you find in `vanity_notifyr.notifyr_privkey_hex` you can create your own notifyr like so:
```python
new_vanity_notifyr = notifyr(gen_vanity_pubkey, recipient_address=your_address)
```

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()